## Resegmentation of grid manager

In [3]:
import os
from pathlib import Path
from grid_manager import GridManager

data_dir = os.path.join("grids", "with-is-residential")
target_dir = os.path.join("grids", "with-is-residentia-resegmented")

target_segment_h = 500
target_segment_w = 500

folder = Path(data_dir)
targets = [(f.name, GridManager(f.name, data_dir=data_dir)) for f in folder.iterdir() if f.is_file()]
print(f"files: {[filename for filename, manager in targets]}")

files: ['Lublin-Polska.city_grid', 'Toruń-Polska.city_grid', 'Gdynia-Polska.city_grid', 'Chorzów-Polska.city_grid', 'Rybnik-Polska.city_grid', 'Koszalin-Polska.city_grid', 'Bydgoszcz-Polska.city_grid', 'Radom-Polska.city_grid', 'Świętochłowice-Polska.city_grid', 'Gdańsk-Polska.city_grid', 'Mielec-Polska.city_grid']


In [ ]:
for filename, grid_manager in targets:

    metadata = grid_manager.get_metadata()

    result = GridManager(
        filename,
        metadata.rows_number,
        metadata.columns_number,
        metadata.upper_left_longitude,
        metadata.upper_left_latitude,
        metadata.grid_density,
        target_segment_h,
        target_segment_w,
        data_dir=target_dir,
        third_dimension_size=metadata.third_dimension_size
    )

    metadata = grid_manager.get_metadata()

    fragment_row, fragment_col = 2500, 2500
    fragment_height, fragment_width = 1500, 1500

    for segment_row in range(metadata.rows_number // metadata.segment_h):
        for segment_column in range(metadata.columns_number // metadata.segment_w):
            segment_start_row = segment_row * metadata.segment_h
            segment_end_row = min(
                (segment_row + 1) * metadata.segment_h,
                metadata.rows_number
            )
            segment_start_column = segment_column * metadata.segment_w
            segment_end_column = min(
                (segment_column + 1) * metadata.segment_w,
                metadata.rows_number
            )
            copied_h = segment_end_row - segment_start_row
            copied_w = segment_end_column - segment_start_column
            result.write_arbitrary_fragment(
                grid_manager.read_arbitrary_fragment(
                    segment_row * metadata.segment_h,
                    segment_column * metadata.segment_w,
                    copied_h,
                    copied_w,
                ),
                copied_h,
                copied_w
            )